<a href="https://colab.research.google.com/github/LeonardoQZ/handson-ml2/blob/master/CaliforniaGeostats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# California Housing with Geostatistics

In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_california_housing
from pandas.plotting import scatter_matrix

# Load, inspect and cleanup the data

In [ ]:
housing_dataset = fetch_california_housing()
print(housing_dataset['DESCR'])

In [ ]:
source_columns = housing_dataset['feature_names'] + ['target']
housing_df = pd.DataFrame(data= np.c_[housing_dataset['data'], housing_dataset['target']],
                     columns= source_columns)

In [ ]:
housing_df.describe()

In [ ]:
import matplotlib.pyplot as plt
housing_df.plot(kind="scatter", x="Longitude", y="Latitude", alpha=0.4,
    s=housing_df["Population"]/100, label="population", figsize=(10,7),
    c="target", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()
plt.show()

There is clearly clipping in the target data. It would make sense to remove the maximum value from the analysis.

We will re-index to have simple range compatible with downstream transformations.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
housing_df['target'].hist(bins=100, ax=axes[0])
clipped_indexes = housing_df[ housing_df['target'].ge(5) ].index
housing_clipped_df = housing_df.drop(index=clipped_indexes).reset_index()
housing_clipped_df['target'].hist(bins=100, ax=axes[1]);

In [ ]:
housing_clipped_df.info()
housing_clipped_df.describe()

# Define training set with stratified sampling

Before doing anything else let us split the data into train and test set. What kind of sampling to use is a non trivial question. There is high correlation between the median income and the target price. 

In [ ]:
scatter_matrix(housing_clipped_df[['MedInc', 'target']], figsize=(12, 8));

We want to guarantee that the test set is representative of the income distribution. In practical terms we would want estimation errors from populations of very high income have acceptable confidence intervals. To achieve this we can stratify our sampling by breaking up the dataset into discrete median income categories. The estimation error of statistics on each of this categories will be smaller if we randomly sample independently from them than if we randomly sample without stratification.

Let us compare strata by Geron, Pew Research with proportional quantile and decile stratification to quantify estimation errors.

In [ ]:
income_category_geron, income_geron_bins = pd.cut(housing_clipped_df["MedInc"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf], retbins=True,
                               labels=[1, 2, 3, 4, 5])
income_category_pew, income_pew_bins = pd.cut(housing_clipped_df["MedInc"],
                               bins=[0., 3.1, 4.2, 12.6, 18.8, np.inf], retbins=True,
                               labels=[1, 2, 3, 4, 5])
income_category_quartiles, income_quartile_bins = pd.qcut(
                               housing_clipped_df["MedInc"], 4, retbins=True,
                               labels=[str(i+1) for i in range(4)])
income_category_deciles, income_decile_bins = pd.qcut(
                               housing_clipped_df["MedInc"], 10, retbins=True,
                               labels=[str(i+1) for i in range(10)])

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 6))
income_category_geron.hist(bins=10, ax=axes[0,0])
income_category_pew.hist(bins=10, ax=axes[0,1])
income_category_quartiles.hist(bins=10, ax=axes[1,0])
income_category_deciles.hist(bins=10, ax=axes[1,1])
plt.show()

In [ ]:
income_categories = pd.DataFrame({
    'Geron' : income_category_geron,
    'Pew' : income_category_pew,
    'Quartiles' : income_category_quartiles,
    'Deciles' : income_category_deciles
})
income_categories.describe()

In [ ]:
housing_categorized = housing_clipped_df.copy()
housing_categorized[income_categories.keys()] = income_categories


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

train_set_random, test_set_random = train_test_split(housing_categorized, test_size=0.2, random_state=42)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [ ]:
train_sets = {"Random" : train_set_random }
test_sets = {"Random" : test_set_random }
for key in income_categories:
  for train_index, test_index in split.split(housing_categorized, housing_categorized[key]):
    train_sets[key] =  housing_categorized.loc[train_index]
    test_sets[key] = housing_categorized.loc[test_index]


In [ ]:
def generateCategoryComparison(income_categories):
  comparison = {}
  for cat in income_categories:
    overall = f"Overall {cat}"
    random = f"Random {cat}"
    stratified = f"Stratified {cat}"
    randErr = f"RandErr {cat}"
    stratErr = f"StratErr {cat}"
    compare_props = pd.DataFrame({
        overall : housing_categorized[cat].value_counts() / len(housing_categorized),
        random : train_sets['Random'][cat].value_counts() / len(train_sets['Random']),
        stratified : train_sets[cat][cat].value_counts() / len(train_sets[cat])
    }).sort_index()
    compare_props[randErr] = 100*np.abs(compare_props[random]/compare_props[overall] - 1)
    compare_props[stratErr] = 100*np.abs(compare_props[stratified]/compare_props[overall] - 1)
    comparison[cat] = compare_props
  return comparison

In [ ]:
cat_comparison = generateCategoryComparison(income_categories)


In [ ]:
cat_comparison

The Income stratification model of Pew Research gives the lowest sampling bias errors but on this particular dataset there are no units with a median in category 5 and very fey in 4. Let us adapt a bit the category.

In [ ]:
income_strata, income_strata_bins = pd.cut(housing_clipped_df["MedInc"],
                               bins=[0., 3.1, 4.2, 7.5, 9.3, np.inf], retbins=True,
                               labels=[1, 2, 3, 4, 5])
housing_categorized["Strata"] = income_strata
train_set_random, test_set_random = train_test_split(housing_categorized, test_size=0.2, random_state=42)
train_sets['Random']=train_set_random
test_sets['Random']=test_set_random

In [ ]:
for train_index, test_index in split.split(housing_categorized, housing_categorized["Strata"]):
    train_sets["Strata"] =  housing_categorized.loc[train_index]
    test_sets["Strata"] = housing_categorized.loc[test_index]

In [ ]:
cat_comparison_final = generateCategoryComparison(['Pew', 'Strata'])

In [ ]:
cat_comparison_final

In [ ]:
income_strata.value_counts()

This seems a good starting point for our train and test dataset. Let us make a copy with the categories removed.

In [ ]:
train_set = train_sets['Strata'].copy()
test_set = test_sets['Strata'].copy()
train_set.drop(income_categories, axis=1, inplace=True)
train_set.drop('Strata', axis=1, inplace=True)
test_set.drop(income_categories, axis=1, inplace=True)
test_set.drop('Strata', axis=1, inplace=True)


# Visualization

Let us get a birds eye view of the correlations in the data

In [ ]:

scatter_matrix(train_set, figsize=(12, 8));

In [ ]:
correlations = train_set.corr()
print(correlations["target"].sort_values(ascending=False))

Maybe combining some of the features in a sensible way we can find other correlations

In [ ]:
rooms_per_capita = train_set['AveRooms']/train_set['Population']
bedroom_ratio = train_set['AveBedrms']/train_set['AveRooms']


In [ ]:
correlations_xtra = housing_df.corr()
print(correlations_xtra["target"].sort_values(ascending=False))

corr_threshold = 0.15
high_corr = correlations_xtra["target"].sort_values(ascending=False).abs().ge(corr_threshold)
high_corr_cols=list(high_corr[high_corr].keys())

In [ ]:
scatter_matrix(housing_df[high_corr_cols], figsize=(12, 8));

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
